In [ ]:
# import libraries
!pip install datasets
!pip install --upgrade huggingface-hub
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
import random
import pandas as pd
# Load sentiment analysis dataset from Hugging Face datasets library
dataset = load_dataset("sentiment140")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])


In [ ]:
df['sentiment'] = df['sentiment'].replace(4,1)

In [ ]:
# Filter dataset to ensure all target classes are represented
class_0_data = df[df['sentiment'] == 0].sample(100)
class_1_data = df[df['sentiment'] == 1].sample(100)

In [ ]:
subset_df = pd.concat([class_0_data, class_1_data])


In [ ]:
# Convert back to dataset dict
subset_dataset = {
    'train': {
        'text': subset_df['text'].tolist(),
        'sentiment': subset_df['sentiment'].tolist()
    }
}

In [ ]:
# Convert dataset dict to PyTorch tensors
input_texts = subset_dataset['train']['text']
labels = torch.tensor(subset_dataset['train']['sentiment'])

# Define the BERT model
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input text and add special tokens
input_ids = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')['input_ids']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Create DataLoader
data_set = TensorDataset(input_ids, labels)
data_loader = DataLoader(data_set, batch_size=4, shuffle=True)

# Fine-tuning BERT
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Move input tensors to the device
##inputs = inputs.to(device)
#labels = labels.to(device)

# Move the model to the device
bert_model = bert_model.to(device)

for epoch in range(3):
    bert_model.train()
    total_loss = 0
    for batch in data_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs, labels = batch

        optimizer.zero_grad()

        outputs = bert_model(inputs, labels=labels)
        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch {} Loss: {}".format(epoch+1, total_loss))


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1 Loss: 35.26321363449097
Epoch 2 Loss: 32.02982971072197
Epoch 3 Loss: 19.72529000788927


In [ ]:
# Save the fine-tuned model
bert_model.save_pretrained("refined_bert")

print("Fine-tuning completed and model saved.")


Fine-tuning completed and model saved.


In [ ]:
# Load IMDb dataset from Hugging Face datasets library
imdb_dataset = load_dataset("imdb")

# Convert evaluation dataset to pandas DataFrame
imdb_df = pd.DataFrame(imdb_dataset['train'])

# Filter dataset to ensure both target classes are represented
class_0_data = imdb_df[imdb_df['label'] == 0].sample(50)
class_1_data = imdb_df[imdb_df['label'] == 1].sample(50)
subset_imdb_df = pd.concat([class_0_data, class_1_data])

# Convert back to dataset dict
subset_imdb_dataset = {
    'train': {
        'text': subset_imdb_df['text'].tolist(),
        'label': subset_imdb_df['label'].tolist()
    }
}

# Convert dataset dict to PyTorch tensors
eval_texts = subset_imdb_dataset['train']['text']
eval_labels = torch.tensor(subset_imdb_dataset['train']['label'])

# Load fine-tuned BERT model
bert_model = BertForSequenceClassification.from_pretrained('refined_bert')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Tokenize evaluation text and add special tokens
eval_input_ids = tokenizer(eval_texts, padding=True, truncation=True, return_tensors='pt')['input_ids']

# Create DataLoader
eval_data_set = TensorDataset(eval_input_ids, eval_labels)
eval_data_loader = DataLoader(eval_data_set, batch_size=1)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Prediction function
def predict(model, loader):
    predictions = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs[0], 1)
            predictions.append(predicted.item())
    return predictions

# Predict labels
predicted_labels = predict(bert_model, eval_data_loader)

print("Predicted Labels:", predicted_labels)

Predicted Labels: [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
# True labels
true_labels = eval_labels.numpy()

eval_labels_np = eval_labels.numpy()
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Calculate classification report
class_report = classification_report(eval_labels_np, predicted_labels)

# Calculate accuracy
accuracy = accuracy_score(eval_labels_np, predicted_labels)

# Calculate F1 score
f1score = f1_score(eval_labels_np, predicted_labels, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print("\nAccuracy:", accuracy)
print("\nF1 Score:", f1score)

Confusion Matrix:
[[ 7 43]
 [ 1 49]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.14      0.24        50
           1       0.53      0.98      0.69        50

    accuracy                           0.56       100
   macro avg       0.70      0.56      0.47       100
weighted avg       0.70      0.56      0.47       100


Accuracy: 0.56

F1 Score: 0.46576007770762506
